#MIDTERM-PArtB-ProgrammingQuestion
##Author - Rishika Nidiganti

In this report, I have used Cassandra to support the requirements that were asked. The new feature that is being introduced to the existing chat application is file attachment. This feature allows the users to attach the files and these files are stored to the database along with the username, the date that the file is attached, the type of the file and the attached file. 

I used Cassandra to store this information. The process is as follows-

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

Connecting to the 3-node cluster, which I started in the terminal

In [2]:
clstr = Cluster()
session = clstr.connect()

Dropping any keyspaces if already exists

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [ ]:
Checking to see all the available keyspaces

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


Creating a keyspace called m14 with a replication factor of 2, as it is a 3 node cluster.

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':2
    }
""")

Checking to see if the keyspace has been created

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


Here, I have created a table called fileattachments, which stores all the information like username, date_attachment, file_type and the attached_file

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.fileattachments ( 
    username TEXT, 
    date_attachment TEXT, 
    file_type TEXT, 
    attached_file TEXT, 
    PRIMARY KEY(username, date_attachment)
);
""")

I am inserting some sample made up data into this table

In [8]:
# Sample data to insert
sample_data = [
    ("uname1", "10-04-2023", "txt", "file1.txt"),
    ("uname2", "10-03-2023", "png", "file2.png"),
    ("uname3", "10-04-2023", "txt", "file3.txt"),
    ("uname4", "10-01-2023", "mov", "file4.mov"),
    ("uname5", "10-02-2023", "txt", "file5.txt"),
]

# Insert the data
for entry in sample_data:
    insert_query = """
    INSERT INTO m14.fileattachments (username, date_attachment, file_type, attached_file)
    VALUES (%s, %s, %s, %s)
    """
    session.execute(insert_query, entry)


In [ ]:
In order to retrieve all the stored files from the database, we use the following query:

In [9]:
# Query to fetch all records from fileattachments
query = "SELECT * FROM m14.fileattachments"
rows = session.execute(query)

# Print the data
print("username\t\tDate_attachment\t\tfile_type\t\tfile_attached")
print("-" * 80)  # just a separator line
for row in rows:
    print(f"{row.username}\t\t{row.date_attachment}\t\t{row.file_type}\t\t{row.attached_file}")


username		Date_attachment		file_type		file_attached
--------------------------------------------------------------------------------
uname3		10-04-2023		txt		file3.txt
uname2		10-03-2023		png		file2.png
uname5		10-02-2023		txt		file5.txt
uname4		10-01-2023		mov		file4.mov
uname1		10-04-2023		txt		file1.txt


In [10]:
rows = session.execute("SELECT (username, date_attachment, file_type, attached_file) FROM m14.fileattachments")
for row in rows:
    print(f"username={row[0][0]}, date_attachment={row[0][1]}, file_type={row[0][2]}, attached_file={row[0][3]}")

username=uname3, date_attachment=10-04-2023, file_type=txt, attached_file=file3.txt
username=uname2, date_attachment=10-03-2023, file_type=png, attached_file=file2.png
username=uname5, date_attachment=10-02-2023, file_type=txt, attached_file=file5.txt
username=uname4, date_attachment=10-01-2023, file_type=mov, attached_file=file4.mov
username=uname1, date_attachment=10-04-2023, file_type=txt, attached_file=file1.txt


Now, in order to insert the attchments into the database, we can develop function called add_file() which inserts the attached file into our table.

In order to delete an existing record, we can use a function called delete_file(), which deletes the file from the system

All these new entries will be inserted into our database following the same format. The username will be the username of the person who is attaching the file, the date that he has attached the file, the type of the file that he is attaching and the attached file will be stored in the database.

In this way users can now attach files or delete them if not needed. The files can also be retrieved as required.